In [1]:
%%capture

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from datetime import datetime
date_du_jour = datetime.today().strftime("%d_%m_%Y")
from pandas import DataFrame, ExcelWriter
import openpyxl


from caregiver import id_parent_served, parents, sdata, id_parent_completed
from handler import name_handler

In [2]:

id_parent_served.drop_duplicates('id_patient',inplace=True)
id_parent_completed.drop_duplicates('id_patient',inplace=True)
parents.drop_duplicates('code',inplace=True)
sdata.drop_duplicates('case_id',inplace=True)

/var/folders/7v/rs0k237d6nj3vdxfc1h7xtcw0000gn/T/ipykernel_19254/1617343120.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  id_parent_served.drop_duplicates('id_patient',inplace=True)
/var/folders/7v/rs0k237d6nj3vdxfc1h7xtcw0000gn/T/ipykernel_19254/1617343120.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  id_parent_completed.drop_duplicates('id_patient',inplace=True)
/var/folders/7v/rs0k237d6nj3vdxfc1h7xtcw0000gn/T/ipykernel_19254/1617343120.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

In [3]:
id_parent_served.id_patient.count()
id_parent_completed.id_patient.count()
parents.code.count()
sdata.case_id.count()



235

101

486

70374

In [4]:
lc_idparentserved = sdata[sdata.id_patient.isin(id_parent_served.id_patient)]
lc_ips = lc_idparentserved[['id_patient','code']]
lc_idparentserved.code.count()
lc_ips.code.count()

235

235

In [5]:
code_in_parents = lc_ips[lc_ips.code.isin(parents.code)]
code_notIn_parents = lc_ips[~lc_ips.code.isin(parents.code)]


code_in_parents.code.count()
code_notIn_parents.code.count()

214

21

# the completed

In [6]:
id_parent_completed[id_parent_completed.id_patient.isin(code_in_parents.id_patient)].id_patient.count()
id_parent_completed[~id_parent_completed.id_patient.isin(code_in_parents.id_patient)].id_patient.count()



importantMissing = id_parent_completed[~id_parent_completed.id_patient.isin(code_in_parents.id_patient)]
importantMissing = sdata[sdata.id_patient.isin(importantMissing.id_patient)]

97

4


# COnclusion


- fosref must add "Code_notIn_parents" for the caregiver
- code_in_parents are Datim whorty

## not in parents

In [7]:
missing_FAP = sdata[sdata.id_patient.isin(code_notIn_parents.id_patient)]
missing_FAP.id_patient.count()

21

# in parents - datim

In [8]:
found_FAP = sdata[sdata.id_patient.isin(code_in_parents.id_patient)]
found_FAP.id_patient.count()

cip = code_in_parents[['code']]
parents_reportable = cip.merge(parents,how='left',on='code')
parents_reportable.code.count()

214

214

In [9]:
parents_reportable.pivot_table(index="commune",values="code",columns=["Gender","age_ovc"],aggfunc='count',fill_value=0)

Gender,Female,Male
age_ovc,18+,18+
commune,,
cap-hatien,16,8
delmas,2,0
desdunes,20,8
dessalines,8,15
grande-saline,15,0
gressier,14,1
kenscoff,27,14
la_chapelle,1,0


In [10]:
datim = parents_reportable.pivot_table(index="commune",values="code",columns=["Gender","age_ovc"],aggfunc='count',fill_value=0)


In [11]:
datim[('Female','<1')] = 0
datim[('Female','1-4')] = 0
datim[('Female','5-9')] = 0
datim[('Female','10-14')] = 0
datim[('Female','15-17')] = 0
datim[('Male','<1')] = 0
datim[('Male','1-4')] = 0
datim[('Male','5-9')] = 0
datim[('Male','10-14')] = 0
datim[('Male','15-17')] = 0

columns_orientation =[
    ('Female','<1'),
    ('Female','1-4'),
    ('Female','5-9'),
    ('Female','10-14'),
    ('Female','15-17'),   
    ('Female','18+'),  
    ('Male','<1'),
    ('Male','1-4'),
    ('Male','5-9'),
    ('Male','10-14'),
    ('Male','15-17'),   
    ('Male','18+'),   
]

datim = datim.reindex(columns= columns_orientation)
datim

Gender                       Female                         Male          \
age_ovc                          <1 1-4 5-9 10-14 15-17 18+   <1 1-4 5-9   
commune                                                                    
cap-hatien                        0   0   0     0     0  16    0   0   0   
delmas                            0   0   0     0     0   2    0   0   0   
desdunes                          0   0   0     0     0  20    0   0   0   
dessalines                        0   0   0     0     0   8    0   0   0   
grande-saline                     0   0   0     0     0  15    0   0   0   
gressier                          0   0   0     0     0  14    0   0   0   
kenscoff                          0   0   0     0     0  27    0   0   0   
la_chapelle                       0   0   0     0     0   1    0   0   0   
liancourt                         0   0   0     0     0   9    0   0   0   
limonade                          0   0   0     0     0   4    0   0   0   
montrouis                         0   0   0     0     0   3    0   0   0   
petion_ville                      0   0   0     0     0  10    0   0   0   
petite_rivire_de_lartibonite      0   0   0     0     0   5    0   0   0   
tabarre                           0   0   0     0     0  19    0   0   0   
verrettes                         0   0   0     0     0   5    0   0   0   

Gender                                        
age_ovc                      10-14 15-17 18+  
commune                                       
cap-hatien                       0     0   8  
delmas                           0     0   0  
desdunes                         0     0   8  
dessalines                       0     0  15  
grande-saline                    0     0   0  
gressier                         0     0   1  
kenscoff                         0     0  14  
la_chapelle                      0     0   0  
liancourt                        0     0   0  
limonade                         0     0   1  
montrouis                        0     0   2  
petion_ville                     0     0   1  
petite_rivire_de_lartibonite     0     0   1  
tabarre                          0     0   5  
verrettes                        0     0   0

# REPORT

In [12]:
#datim.to_excel(f"./caregiver_results/Q2/datim_parenting_caregiver{date_du_jour}.xlsx",na_rep="",index=True)
#found_FAP.to_excel(f"./caregiver_results/Q2/datim_caregiver_{date_du_jour}.xlsx",na_rep="",index=False)

#datim.to_excel(f"./caregiver_results/Q3/datim_parenting_caregiver{date_du_jour}.xlsx",na_rep="",index=True)
#found_FAP.to_excel(f"./caregiver_results/Q3/datim_caregiver_{date_du_jour}.xlsx",na_rep="",index=False)


found_FAP.to_excel(f"./caregiver_results/datim_caregiver.xlsx",na_rep="",index=False)
datim.to_excel(f"./caregiver_results/datim_parenting_caregiver.xlsx",na_rep="",index=True)

# no Datim

In [13]:
missing_FAP_communes = []
for columns in missing_FAP.commune.unique():
    globals()[f"missing_FAP_{name_handler(columns)}"] = missing_FAP[missing_FAP.commune == columns]
    missing_FAP_communes.append(globals().get(f"missing_FAP_{name_handler(columns)}"))
    

importantMissing_communes = []
for columns in importantMissing.commune.unique():
    globals()[f"importantMissing_{name_handler(columns)}"] = importantMissing[importantMissing.commune == columns]
    importantMissing_communes.append(globals().get(f"importantMissing_{name_handler(columns)}"))
    


In [14]:
mfap_communes = missing_FAP.commune.unique().tolist()
missing_FAP_dict = {}
for columns, communes in zip(missing_FAP.commune.unique(), mfap_communes):
    missing_FAP_dict[f"{name_handler(communes)}"] =  globals().get(f"missing_FAP_{name_handler(columns)}").code.count()
        
missing_FAP_df = DataFrame.from_dict(missing_FAP_dict, orient="index", columns=None)
missing_FAP_df.columns = ["nombre_agyw/caregiver_KAP_pour_Regularisation"]
missing_FAP_df



im_communes = importantMissing.commune.unique().tolist()
importantMissing_dict = {}
for columns, communes in zip(importantMissing.commune.unique(), im_communes):
    importantMissing_dict[f"{name_handler(communes)}"] =  globals().get(f"importantMissing_{name_handler(columns)}").code.count()
    
importantMissing_df = DataFrame.from_dict(importantMissing_dict, orient="index", columns=None)
importantMissing_df.columns = ["nombre_agyw/caregiver_KAP_pour_Regularisation"]
importantMissing_df

,nombre_agyw/caregiver_KAP_pour_Regularisation
kenscoff,5
tabarre,4
verrettes,1
caphatien,2
dessalines,5
desdunes,2
montrouis,1
grandesaline,1


,nombre_agyw/caregiver_KAP_pour_Regularisation
kenscoff,2
desdunes,2


In [15]:
importantMissing.to_excel('./missing/caregiver_CP_missing.xlsx',index=False,na_rep="")

In [16]:
missfap = ExcelWriter(f"./missing/missing_FAP_{date_du_jour}.xlsx",engine="openpyxl")
missing_FAP_df.to_excel(missfap,sheet_name='Resume',index=True)
missing_FAP.to_excel(missfap,sheet_name="Total", na_rep="",index=False)
for columns in missing_FAP.commune.unique():
    globals().get(f"missing_FAP_{name_handler(columns)}").to_excel(missfap,sheet_name=f"{name_handler(columns)}",index=False,na_rep="NULL")
missfap.save()

In [17]:
IM = ExcelWriter(f"./missing/caregiver_CP_missing.xlsx",engine="openpyxl")
importantMissing_df.to_excel(IM,sheet_name='Resume',index=True)
importantMissing.to_excel(IM,sheet_name="Total", na_rep="",index=False)
for columns in importantMissing.commune.unique():
    globals().get(f"importantMissing_{name_handler(columns)}").to_excel(IM,sheet_name=f"{name_handler(columns)}",index=False,na_rep="NULL")
IM.save()